In [87]:
import arcpy
import os

In [26]:
# pwd  #default your workspace is in your project

# Get current directory path
current_dir = os.getcwd()

# List all files and folders in the current directory
files_and_folders = os.listdir(current_dir)

# print(files_and_folders)

current_dir

'C:\\WINDOWS\\System32'

In [10]:
# arcpy.management.CopyFeatures("AffectedAreaApp", r"D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb\AffectedAreaApp_CopyFeatures",
#                               '', None, None, None)


## see all data inside a geodatabase

In [27]:
arcpy.env.workspace = r"D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb"
fclasses = arcpy.ListFeatureClasses()
fclasses

['ParcelPts', 'ParcelPoly', 'ServiceAreas']

## It's a nice way to check if any layers have missing,  GCS possibly use a if else statement to see them separately

In [28]:
for fc in fclasses:
    desc = arcpy.Describe(fc)
    spatial_ref = desc.spatialReference.name  # Get the spatial reference name
    print(f"Feature Class: {fc}, Type: {desc.shapeType}, Spatial Reference: {spatial_ref}")

Feature Class: ParcelPts, Type: Point, Spatial Reference: NAD_1983_CORS96_StatePlane_North_Carolina_FIPS_3200_Ft_US
Feature Class: ParcelPoly, Type: Polygon, Spatial Reference: NAD_1983_CORS96_StatePlane_North_Carolina_FIPS_3200_Ft_US
Feature Class: ServiceAreas, Type: Polygon, Spatial Reference: NAD_1983_CORS96_StatePlane_North_Carolina_FIPS_3200_Ft_US


## see all geodatabases in a parent directory and data inside them


In [31]:


# Set the parent directory containing multiple geodatabases
parent_folder = r"D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data"

# List to store feature classes from all geodatabases
all_feature_classes = []

# Walk through the directory to find all geodatabases
for root, dirs, files in os.walk(parent_folder):
    for directory in dirs:
        if directory.endswith(".gdb"):
            gdb_path = os.path.join(root, directory)
            arcpy.env.workspace = gdb_path
            
            # List all feature classes in the current geodatabase
            fclasses = arcpy.ListFeatureClasses()
            print(fclasses)
            if fclasses:
                all_feature_classes.extend([(directory, fc) for fc in fclasses])

# Print the collected feature classes
for gdb, feature_class in all_feature_classes:
    print(f"GDB: {gdb}\n Feature Class: {feature_class}")


['ParcelPts', 'ParcelPoly', 'ServiceAreas']
['AllOutages']
GDB: CountyData.gdb
 Feature Class: ParcelPts
GDB: CountyData.gdb
 Feature Class: ParcelPoly
GDB: CountyData.gdb
 Feature Class: ServiceAreas
GDB: Outages.gdb
 Feature Class: AllOutages


In [11]:
 arcpy.ListFeatureClasses("", "Point")

['ParcelPts']

In [14]:
arcpy.ListFields("ParcelPts")[0].name

'OBJECTID'

In [15]:
for fields in arcpy.ListFields("ParcelPts"):
    print(fields.name)

OBJECTID
Shape
Parcel_ID
Street_Number
Street_Name
City
State
Zip
Owner_Name
Phone_Number
SquFoot
TaxValue
PriceSquFt


## Iterating over all the point feature class and exporting them in the designated gdb

In [29]:
outWorkspace = r"D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb"
fclist =arcpy.ListFeatureClasses("", "Point")
for fc in fclist:
    outputfc = f"{outWorkspace}\\{fc}"
    arcpy.CopyFeatures_management (fc, outputfc) #(input feature, output feature)
    

## Now iterate over all feature classes in a parent folder and export them in a new gdb

In [35]:
import arcpy
import os

# Define parent folder containing geodatabases and the target output geodatabase
parent_folder = r"D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data"
out_workspace = r"D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb"

# Loop through all geodatabases in the parent folder
for root, dirs, files in os.walk(parent_folder):
    for directory in dirs:
        if directory.endswith(".gdb"):
            gdb_path = os.path.join(root, directory)
            arcpy.env.workspace = gdb_path
            
            # List all point feature classes in the current geodatabase
            fclist = arcpy.ListFeatureClasses()
            
            if fclist:
                for fc in fclist:
                    # Define the output feature class path
                    output_fc = f"{out_workspace}\\{fc}"
                    
                    # Copy the feature class to the target geodatabase
                    arcpy.CopyFeatures_management(fc, output_fc)
                    print(f"Copied {fc} from {gdb_path} to {output_fc}")

print("Copy operation completed.")



Copied ParcelPts from D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb to D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb\ParcelPts
Copied ParcelPoly from D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb to D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb\ParcelPoly
Copied ServiceAreas from D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb to D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb\ServiceAreas
Copied AllOutages from D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\Outages.gdb to D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb\AllOutages
Copy operation completed.


## Cursor Search to get values inside a column or to get rows

- when working with ArcGIS Python cursors (like SearchCursor and UpdateCursor) to access data in a database, you should always use a "with" statement to manage the cursor, which automatically resets the iteration and helps release database locks when the loop finishes; 
- additionally, to ensure the cursor is fully released and prevent potential locking issues, explicitly delete the cursor object using the "`del`" keyword or wrap the cursor creation within a function to automatically manage its scope

In [32]:
arcpy.env.workspace = r"D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb"
# instead of working with original geodatabase now i am working with copied data
# arcpy.env.workspace =r"D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb"
 
feature1 = arcpy.ListFeatureClasses ()[3]
feature1

'ParcelPts'

In [33]:
uFields = ["SquFoot", "TaxValue", "PriceSquFt"] 
with arcpy.da.UpdateCursor('ParcelPts', uFields) as uCursor:
    for row in uCursor:
        row[2] = row[1] / row[0]
        uCursor.updateRow(row)                           # here we have 3 columns (Ufields) within the ucursor
del uCursor                                              # row[0] is SqFoot column and accessing all rows for that 
                                                        # column, row[2] is actually an empty field created to fill this way.

In [34]:

# Define SQL query for selection
sql_cmnd = "TaxValue BETWEEN 172056.0 AND 338470.0"

# Use a SearchCursor to access data and limit the output to 10 records
with arcpy.da.SearchCursor('ParcelPts', ["Owner_Name", "TaxValue"], sql_cmnd) as cursor:
    for i, row in enumerate(cursor):
        print(row[0], row[1])
        if i == 9:  # Stop after 10 records
            break
del cursor

Keneth Syers 172056.0
Laurette Albritton 188640.0
Alease Howk 264616.0
Erika Beaven 338470.0
Lowell Pilkins 273136.0
Magali Yelle 292236.0
Shanita Jeng 180937.0
Meta Cheas 245708.0
Caprice Gabrysch 196226.0
Shemeka Kinas 289575.0


## deleting those rows,for demonstration purpose I am just deleting 10 rows

In [130]:
##copying this fc in anoter location is a good idea before deleting

In [35]:
with arcpy.da.UpdateCursor('ParcelPts', ["Owner_Name", "TaxValue"], sql_cmnd) as cursor:
    count = 0
    for row in cursor:
        cursor.deleteRow()  # Delete the current row
        count += 1
        if count == 10:  # Stop after deleting 10 rows
            break
del cursor

print(f"{count} rows deleted.")

10 rows deleted.


In [129]:


# Set workspace and path to the feature class
arcpy.env.workspace = r"D:\GIS\for PRO\Arcgis_Python_training\Python Scripting with ArcGIS Pro 2208\PYTS_8.0_Aug22_StudentData\PYTS\Data\CountyData.gdb"
feature_class = 'ParcelPts'

# Define SQL query for selection
sql_cmnd = "TaxValue BETWEEN 28356 AND 338470.0"

# Create an Editor object to ensure proper transaction handling, it only deletes if all records are deleted,
# no partial delete

workspace_path = arcpy.env.workspace
with arcpy.da.Editor(workspace_path) as editor:  # Automatically starts an edit session
    try:
        with arcpy.da.UpdateCursor(feature_class, ["Owner_Name", "TaxValue"], sql_cmnd) as cursor:
            for row in cursor:
                cursor.deleteRow()
        
        print("All matching rows deleted successfully.")
    except Exception as e:
        print(f"Error during row deletion: {e}")



All matching rows deleted successfully.


## let's convert our featureclass into a df



In [88]:
import pandas as pd
# import seaborn as sns
from arcgis.gis import GIS

df = pd.DataFrame.spatial.from_featureclass(feature1)
df.head()



,OBJECTID,Parcel_ID,Street_Number,Street_Name,City,State,Zip,Owner_Name,Phone_Number,SquFoot,TaxValue,PriceSquFt,SHAPE
0,10,17999752,3516,Laura Ray Road,Linden,NC,28356,Jonnie Kirkness,990-585-8885,2118.0,171558.0,81.0,"{""x"": 2058555.1953131407, ""y"": 537545.38245105..."
1,11,17999753,7597,Big Bay Road,Linden,NC,28356,Sherilyn Fuente,990-447-9005,1254.0,106590.0,85.0,"{""x"": 2058719.3193287253, ""y"": 537758.88103981..."
2,13,17999759,6109,Giles Road,Linden,NC,28356,Phuong Marcelin,990-215-6570,3377.0,327569.0,97.0,"{""x"": 2058268.4304667264, ""y"": 538604.09686906..."
3,14,17999760,7819,Carlos Road,Linden,NC,28356,Tracee Corbisiero,990-915-9679,1845.0,190035.0,103.0,"{""x"": 2058492.283693552, ""y"": 538065.196700140..."
4,15,17999761,4011,Herrill Baggett Road,Linden,NC,28356,Sherilyn Scafuto,990-614-7432,2720.0,293760.0,108.0,"{""x"": 2058455.6272708029, ""y"": 538188.43333031..."


In [89]:

df.isna().sum()

OBJECTID         0
Parcel_ID        0
Street_Number    0
Street_Name      0
City             0
State            0
Zip              0
Owner_Name       0
Phone_Number     0
SquFoot          0
TaxValue         0
PriceSquFt       0
SHAPE            0
dtype: int64

In [90]:
uplicates = df[df.duplicated(subset=["Owner_Name"], keep=False)] 
uplicates

,OBJECTID,Parcel_ID,Street_Number,Street_Name,City,State,Zip,Owner_Name,Phone_Number,SquFoot,TaxValue,PriceSquFt,SHAPE


In [92]:
# you have to import arcgis before this
gis = GIS (profile ="agol_profile", verify_cert = False)
df.spatial.plot()

Profile agol_profile does not have a password on file through keyring. If you are expecting this behavior (PKI or IWA authentication, entering password through run-time prompt, etc.), please ignore this message. If you would like to store your password in the agol_profile profile, run GIS(profile = 'agol_profile', password = ...). See the API doc for more details. (http://bit.ly/2CK2wG8)


MapView(layout=Layout(height='400px', width='100%'))

In [93]:
m = gis.map("United States")
m.zoom = 3
m

MapView(layout=Layout(height='400px', width='100%'))

In [94]:
df.spatial.plot( renderer_type ="u",
               col ="TaxValue")

MapView(layout=Layout(height='400px', width='100%'))

In [95]:
m2 = gis.map("United States")
# m2.center =  [39, -98]
# m2.zoom = 3
m2.legend =True
m2

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [96]:
df.spatial.plot(map_widget =m2,
                renderer_type ="c", #classbreak renderer
                method = "esriClassifyNaturalBreaks",
                class_count = 5,
                alpha =.8, #specify opacity
               col ="TaxValue")


True

now u can manipulate this df the way u want after that u can either upload 
it in arcgis online as a feature layer.
or u can  run the copyfeatures tool/directly to put it
back to a local gdb as a new fc.

## To download features from a folder in ArcGIS Online


In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "your_username", "your_password")

# Specify your ArcGIS Online folder name
folder_name = "MyFolder"
target_gdb = r"D:\GIS\for PRO\Arcgis_Python_training\Python_Arcgis\Python_Arcgis.gdb"

# Search for feature layers in the folder
user = gis.users.me
items = user.items(folder=folder_name, item_type="Feature Layer")

print(f"Found {len(items)} feature layers in the folder '{folder_name}'.")

# Download and copy features to the local geodatabase
for item in items:
    print(f"Processing {item.title}...")
    feature_layer_collection = FeatureLayerCollection.fromitem(item)

    # Export each layer to a geodatabase-compatible format
    feature_layers = feature_layer_collection.layers
    for layer in feature_layers:
        layer_name = layer.properties.name.replace(" ", "_")
        output_fc = f"{target_gdb}\\{layer_name}"

        # Convert the feature set to a spatial DataFrame and save it locally
        sdf = layer.query().sdf
        if not sdf.empty:
            sdf.spatial.to_featureclass(location=output_fc)
            print(f"Copied {layer_name} to {output_fc}")
        else:
            print(f"No features found in {layer_name}.")

print("Copy operation completed.")
